
# Typangaben (Type Hints) in Python – eine praxisnahe Einführung


## Lernziele
- Verstehen, was Typangaben sind und **was sie nicht** sind (keine Laufzeit‑Erzwingung).
- Typen korrekt **annotieren**: Variablen, Funktionen, Klassen (Attribute, Methoden).
- Wichtige Bausteine aus `typing`: `Union`/`|`, `Optional`, `Literal`, `Annotated`, `Callable`, `Iterable`, `Mapping`, `Sequence`, `TypedDict`, `Protocol`, `TypeVar`/Generics, `Final`, `ClassVar`, `Self`.
- Unterschiede zwischen **statischer Prüfung** (Pylance) und **Laufzeit**.
- Typangaben in Klassen aus der Unterrichtseinheit **"Klassen"** anwenden.
- In einer **umfangreichen Übung** ein kleines, gut typisiertes Modul entwickeln.



## 1) Was sind Typangaben?

- **Type Hints** wurden mit [PEP 484] eingeführt und sind **Metadaten**: Sie beschreiben, welche Typen beabsichtigt sind.  
- **Wichtig:** Python **erzwingt** diese Typen **nicht** automatisch zur Laufzeit.  
- Nutzen:
  - Bessere Lesbarkeit und Wartbarkeit.
  - **Statische Analyse** findet Fehler früh (z. B. in VS Code/Pylance). 
  - Bessere Autovervollständigung in IDEs.

**Merke:** „Grüner Check“ vom Type Checker garantiert keine richtige Logik – er minimiert nur eine Klasse von Fehlern.


In [1]:
# 2) Minimales Beispiel: Funktion mit Typangaben

def add(a: int, b: int) -> int:
    return a + b

print(add(2, 3))
# Type Checker würde warnen:
# add("2", 3)   # -> statischer Fehler (Argument 1: erwartet int, erhalten str)


5



## 3) Typ-Syntax ab Python 3.10/3.11

- `Union[int, str]` kann als `int | str` geschrieben werden.
- `Optional[T]` ist Kurzform für `T | None`.
- `list[int]`, `dict[str, int]` usw. (PEP 585: eingebaute generische Typen).
- `from __future__ import annotations` ist **nicht mehr nötig** ab 3.11 für die meisten Fälle.

### Variablen


In [ ]:
# Variablen annotieren
name: str = "Alice"
alter: int = 17
pi: float = 3.14159
werte: list[int] = [1, 2, 3]



### Funktionen


In [ ]:
from collections.abc import Iterable

def mittelwert(xs: Iterable[float]) -> float:
    total = 0.0
    n = 0
    for x in xs:
        total += x
        n += 1
    return total / n if n else float("nan")

print(mittelwert([1.0, 2.0, 3.0]))



### Klassen: Attribute, `Self`, `ClassVar`, `Final`


In [ ]:
from dataclasses import dataclass
from typing import ClassVar, Final, Self

@dataclass
class Konto:
    inhaber: str
    _saldo: float = 0.0

    # Klassenkonstanten / -attribute
    waehrung: ClassVar[str] = "EUR"
    kontoart: Final[str] = "Giro"  # pro Instanz „finale“ Konstante

    def einzahlen(self, betrag: float) -> Self:
        self._saldo += betrag
        return self

    def abheben(self, betrag: float) -> Self:
        if betrag > self._saldo:
            raise ValueError("Zu wenig Guthaben.")
        self._saldo -= betrag
        return self

    @property
    def saldo(self) -> float:
        return self._saldo

k = Konto("Kim").einzahlen(100).abheben(40)
print(k.saldo, Konto.waehrung, k.kontoart)



## 4) Wichtige `typing`-Bausteine im Überblick

### `Union` / `|` und `Optional`


In [ ]:
from typing import Optional

def parse_int(s: str | None) -> Optional[int]:
    if s is None:
        return None
    try:
        return int(s)
    except ValueError:
        return None

print(parse_int("12"), parse_int("x"), parse_int(None))



### `Literal` – feste Werte


In [ ]:
from typing import Literal

def schalter(status: Literal["on", "off"]) -> bool:
    return status == "on"

print(schalter("on"))
# schalter("maybe")  # -> statischer Fehler



### `Annotated` – Typ + Zusatzinfo (z. B. für Validierung/Docs)


In [ ]:
from typing import Annotated

PositiveInt = Annotated[int, "x > 0"]

def wurzel(x: PositiveInt) -> float:
    if x <= 0:
        raise ValueError("x muss > 0 sein")
    return x ** 0.5

print(wurzel(16))



### `Callable` – Funktionssignaturen als Typ


In [ ]:
from collections.abc import Callable

def anwenden(x: int, f: Callable[[int], int]) -> int:
    return f(x)

print(anwenden(5, lambda n: n * 2))



### `TypedDict` & `NamedTuple` – strukturierte, „leichte“ Typen


In [ ]:
from typing import TypedDict, NotRequired

class Student(TypedDict):
    name: str
    klasse: str
    email: NotRequired[str]

s: Student = {"name": "Mina", "klasse": "TGI"}
print(s)



### `Protocol` – strukturelle Typen (Duck Typing mit Verträgen)


In [ ]:
from typing import Protocol

class HatFlaeche(Protocol):
    def flaeche(self) -> float: ...

class Rechteck:
    def __init__(self, b: float, h: float):
        self.b = b; self.h = h
    def flaeche(self) -> float:
        return self.b * self.h

class Kreis:
    def __init__(self, r: float):
        self.r = r
    def flaeche(self) -> float:
        from math import pi
        return pi * self.r * self.r

def gesamtflaeche(figuren: list[HatFlaeche]) -> float:
    return sum(f.flaeche() for f in figuren)

print(gesamtflaeche([Rechteck(2,3), Kreis(1)]))



### Generics mit `TypeVar`, `Generic`, `Self`


In [ ]:
from typing import TypeVar, Generic

T = TypeVar("T")

class Stapel(Generic[T]):
    def __init__(self) -> None:
        self._data: list[T] = []
    def push(self, x: T) -> None:
        self._data.append(x)
    def pop(self) -> T:
        return self._data.pop()
    def __repr__(self) -> str:
        return f"Stapel({self._data!r})"

ints = Stapel[int]()
ints.push(1)
ints.push(2)
print(ints.pop())



### `NewType` und `TypeAlias` – semantisch stärkere Typen


In [ ]:
from typing import NewType, TypeAlias

UserId = NewType("UserId", int)
Primzahl: TypeAlias = int

uid = UserId(42)
zahl: Primzahl = 7
print(uid, zahl)



## 5) Statische Prüfung vs. Laufzeit

- **Statische Checker** (z. B. *mypy*, *pyright* in VS Code, *ruff*) analysieren Quelltext **ohne** ihn auszuführen.
- Zur **Laufzeit** sind Type Hints meist nur Metadaten; sie können mit `typing.get_type_hints` gelesen werden.
- Wenn echte Laufzeit-Prüfung gewünscht ist, nutzt man Validierungsbibliotheken (z. B. `pydantic`) oder schreibt Checks selbst.


In [ ]:
# Laufzeit: Type Hints inspizieren
from typing import get_type_hints

print(get_type_hints(mittelwert))



### mypy/pyright/ruff kurz & knapp

- **VS Code**: Python-Extension + *Pylance* (pyright) aktivieren.  
- **CLI**:  
  - `pip install mypy` → `mypy pfad/zum/projekt`
  - `pip install ruff` → `ruff check --select TCH,ANN .` (Regeln für Typen)
- **Konfig**: `mypy.ini`, `pyproject.toml` oder `ruff.toml` verwenden.



## 6) Typangaben in Klassen (Bezug zur Datei *„Klassen“*)

- **Attribute** in `__init__` sollten **annotiert** werden (oder `@dataclass` verwenden).
- **Abgeleitete Eigenschaften** (`@property`) erhalten ebenfalls Rückgabetypen.
- **Klassenattribute** mit `ClassVar[T]` kennzeichnen.
- **Konstanten** mit `Final[T]` markieren.
- Methoden, die die Instanz **zurückgeben**, mit `Self` typisieren (fluenter Stil).


In [ ]:
# Beispiel „Auto“ – kompakt, ohne Typinfos im Code überladen zu lassen
from dataclasses import dataclass
from datetime import date
from typing import ClassVar, Final

@dataclass
class Auto:
    marke: str
    _baujahr: int  # „privat“ per Konvention

    # Klasseninfo & (quasi) Konstante
    fahrzeugart: ClassVar[str] = "PKW"
    max_alter: Final[int] = 50

    @property
    def baujahr(self) -> int:
        return self._baujahr

    @property
    def alter(self) -> int:
        return date.today().year - self._baujahr

car = Auto("Opel", 2018)
print(car.alter, Auto.fahrzeugart)



---

## 7) Übungsaufgabe (umfangreich): **„Kursverwaltung mit Typen“**

### Aufgabe
Entwickeln Sie ein **kleines, typisiertes Modul** für eine Kursverwaltung. Ziel ist, dass ein Type Checker (mypy/pyright) **keine Fehler** meldet.

### Anforderungen
1. **Datenmodell**
   - Definieren Sie `Student` als `TypedDict` mit Feldern: `id: int`, `name: str`, `email: NotRequired[str]`.
   - Definieren Sie `Kurs` als `dataclass` mit Feldern: `titel: str`, `ects: int`, `teilnehmer: list[Student]`.
   - Definieren Sie `KursId = NewType("KursId", int)`.

2. **Funktionen**
   - `anmelden(kurs: Kurs, student: Student) -> None` (verhindert Duplikate anhand `id`).

   - `finde_student(kurs: Kurs, sid: int) -> Student | None`.

   - `schnitt_note(notenschluessel: Mapping[str, float], noten: Iterable[str]) -> float` – benutze `Mapping`, `Iterable`.

3. **Protokoll**
   - Definieren Sie ein `Protocol` `Bewertbar` mit Methode `note() -> float`.
   - Erstellen Sie zwei Klassen, die `Bewertbar` erfüllen (z. B. `Klausur`, `Projekt`).

4. **Generics**
   - Implementieren Sie eine generische `Stapel[T]`-Klasse (oder nutzen Sie die aus dem Beispiel) und zeigen Sie ihre Verwendung mit `Student` und `KursId`.

5. **Konstanten & Klassenattribute**
   - Kennzeichnen Sie sinnvolle Dinge mit `Final` und `ClassVar`.

6. **Dokumentation**
   - Schreiben Sie zu jeder öffentlichen Funktion eine kurze Docstring mit Typ‑Hinweis.

7. **Bonus (optional)**
   - Nutzen Sie `Annotated` für Zusatzinfos (z. B. zulässige Bereiche), oder `Literal` für erlaubte Statuswerte ("aktiv" | "inaktiv").

### Abgabekriterien
- Code **läuft** ohne Fehler.
- **Type Checker** meldet **0** Fehler.
- Kurze **Tests/Demos** im Notebook vorhanden.



### Starter-Code (zum Ausfüllen)


In [ ]:
# === Ihre Lösung hier entwickeln ===
from __future__ import annotations
from dataclasses import dataclass
from typing import NotRequired, TypedDict, NewType, Protocol
from typing import Mapping, Iterable, Final, ClassVar, TypeVar, Generic

# 1) Datenmodell ------------------------------------------------------------
class Student(TypedDict):
    id: int
    name: str
    email: NotRequired[str]

@dataclass
class Kurs:
    titel: str
    ects: int
    teilnehmer: list[Student]

KursId = NewType("KursId", int)

# 2) Funktionen -------------------------------------------------------------
# TODO: anmelden(kurs, student) -> None
# TODO: finde_student(kurs, sid) -> Student | None
# TODO: schnitt_note(notenschluessel, noten) -> float

# 3) Protokoll --------------------------------------------------------------
class Bewertbar(Protocol):
    def note(self) -> float: ...

# TODO: Klassen, die Bewertbar erfüllen (z. B. Klausur, Projekt)

# 4) Generics ---------------------------------------------------------------
T = TypeVar("T")
# TODO: generische Stapel[T]

# 5) Konstanten & Klassenattribute -----------------------------------------
# TODO: sinnvolle Final / ClassVar verwenden

# 6) Kurze Demos/Tests ------------------------------------------------------
# TODO: ein paar Beispiele aufrufen/ausgeben



> **Tipp für die Klasse „Klassen“:** Übertragen Sie die hier gezeigten Muster auf Ihre vorhandenen Klassendateien. Achten Sie auf:
> - Typen für Attribute und Rückgabewerte
> - `property`-Rückgabetypen
> - `Self` für Methoden, die die Instanz zurückgeben
> - `ClassVar` und `Final` korrekt einsetzen



---

## 8) (Optional) Musterlösung – zum Vergleichen

> **Hinweis:** Eine mögliche Lösung – Ihre darf abweichen, solange der Type Checker zufrieden ist und die Funktionalität stimmt.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import NotRequired, TypedDict, NewType, Protocol
from typing import Mapping, Iterable, Final, ClassVar, TypeVar, Generic

class Student(TypedDict):
    id: int
    name: str
    email: NotRequired[str]

@dataclass
class Kurs:
    titel: str
    ects: int
    teilnehmer: list[Student]

    # Beispiel für ClassVar (z. B. Standard‑Maximalgröße eines Kurses)
    STANDARD_MAX: ClassVar[int] = 30

KursId = NewType("KursId", int)

# --- Funktionen ---
def anmelden(kurs: Kurs, student: Student) -> None:
    """Fügt *student* zu *kurs* hinzu, wenn die id noch nicht vorhanden ist."""
    if any(s["id"] == student["id"] for s in kurs.teilnehmer):
        return
    kurs.teilnehmer.append(student)

def finde_student(kurs: Kurs, sid: int) -> Student | None:
    """Sucht einen Student per *sid*."""
    for s in kurs.teilnehmer:
        if s["id"] == sid:
            return s
    return None

def schnitt_note(notenschluessel: Mapping[str, float], noten: Iterable[str]) -> float:
    """Berechnet den Notenschnitt aus Notencodes gemäß *notenschluessel*."""
    werte: list[float] = []
    for n in noten:
        if n in notenschluessel:
            werte.append(notenschluessel[n])
    return sum(werte) / len(werte) if werte else float("nan")

# --- Protocol ---
class Bewertbar(Protocol):
    def note(self) -> float: ...

@dataclass
class Klausur:
    punkte: int
    MAX_PUNKTE: ClassVar[int] = 100
    BONUS: Final[int] = 0

    def note(self) -> float:
        prozent = self.punkte / self.MAX_PUNKTE
        return 1.0 + (1.0 - prozent) * 4.0  # 1.0..5.0

@dataclass
class Projekt:
    bewertungen: list[float]  # 1.0..6.0

    def note(self) -> float:
        return sum(self.bewertungen) / len(self.bewertungen)

# --- Generics ---
T = TypeVar("T")

class Stapel(Generic[T]):
    def __init__(self) -> None:
        self._data: list[T] = []
    def push(self, x: T) -> None:
        self._data.append(x)
    def pop(self) -> T:
        return self._data.pop()
    def __len__(self) -> int:  # nützlich für Tests
        return len(self._data)

# --- Demo ---
notenschluessel = {"A": 1.0, "B": 2.0, "C": 3.0, "D": 4.0, "E": 5.0}

k = Kurs("Informatik", 6, [])
anna: Student = {"id": 1, "name": "Anna"}
ben: Student = {"id": 2, "name": "Ben", "email": "ben@example.com"}

anmelden(k, anna)
anmelden(k, ben)
anmelden(k, anna)  # Duplikat wird ignoriert

print(f"Teilnehmer: {[s['name'] for s in k.teilnehmer]}")
print("Anna?", finde_student(k, 1))
print("Schnitt ABC:", schnitt_note(notenschluessel, ["A", "B", "C"]))

# Protocol-Demo
b1: list[Bewertbar] = [Klausur(80), Projekt([1.3, 1.7, 2.0])]
print("Noten:", [b.note() for b in b1])

# Generics-Demo
stack_ids = Stapel[KursId]()
stack_ids.push(KursId(10))
stack_ids.push(KursId(11))
print("Stack len:", len(stack_ids))



---

## 9) Checkliste & Best Practices

- Schreibe **Rückgabetypen** (auch `-> None`).
- Nutze **abstrakte** Collection-Typen (`Iterable`, `Mapping`, `Sequence`) für **generische** APIs.
- Markiere **Konstanten** mit `Final`, **Klassenattribute** mit `ClassVar`.
- Für **Fluent APIs** `Self` nutzen.
- **Keine** „magischen“ `Any` – so spezifisch typisieren wie sinnvoll.
- **Type Checker** regelmäßig laufen lassen (CI/Pre-commit).

Viel Erfolg! 🚀
